# Chapter IV-I Heating and Cooling System - General / 暖冷房設備 - 全般

## Heating System and Cooling System / 暖房方式及び冷房方式

### 1. Heating System / 暖房方式

The heating system is classified into two types, "a system of continuously heating the entire house" and "a system of heating only the living room".  
In the case of "a system of heating only the living room", a heating facility equipment or a radiator is assumed for each of the "main room" and "other room". Depending on the assumed heating facility equipment etc., the operation method is determined as "continuous operation" or "intermittent operation" for each of the "main room" and "other room". 
Appendix A shows the heating system and the method of determining the driving method according to the type of heating equipment installed.

### 2. Cooling System / 冷房方式

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>